In [ ]:
from backtesting import *
from backtesting.lib import SignalStrategy, TrailingStrategy
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.lib import crossover, resample_apply
from collections import deque
import tensorflow as tf
import tensorflow.keras as keras
import math

In [ ]:
def SIGNAL1():
    return df.signal1

def ATR():
    return df.atr

def Neural_Network(data, path, stats_dict, seq_len):

    # preprocess
    main_df = data.copy()
    for col in main_df.columns:  # go through all of the columns
        col_end = col.rsplit('_', -1)[-1]  # get the last word of the string
        if col == "Volume" or col_end == "volume":
            main_df[col] = (main_df[col].values - stats_dict[f'{col}']['mean']) / (stats_dict[f'{col}']['std'])

        elif col_end == "value":
            main_df[col] = (main_df[col].values - stats[f'{col}']['mean']) / (stats[f'{col}']['std'])

        elif col == "Open" or col == "High" or col == "Low" or col == "Close":
            main_df[col] = (main_df[col].values - stats_dict[f'{col}']['mean']) / (stats_dict[f'{col}']['std'])

    main_df.dropna(inplace=True) 

    main_df.dropna(inplace=True)
    sequential_data = []  
    prev_days = deque(
        maxlen=seq_len)  
    
    for i in main_df.values:  
        prev_days.append([n for n in i[:]]) 
        if len(prev_days) == seq_len:  
            sequential_data.append(np.array(prev_days)) 

    X = []
    for seq in sequential_data:  
        X.append(seq)  

    final_data = np.array(X)

    model = tf.keras.models.load_model(path)
    predictions = model.predict(final_data)

    pred_list = []
    for pred in predictions:
        
        # Buy
        if (pred[2] > pred[0]) and (pred[2] > pred[1]):
            pred_list.append(2)
        # Sell
        elif (pred[0] > pred[2]) and (pred[0] > pred[1]):
            pred_list.append(0)
        # No trade
        else:
            pred_list.append(1)

            
    buy_list = []
    sell_list = []
    flat_list = []

    for pred in predictions:
        
        buy_list.append(pred[2])
        sell_list.append(pred[0])
        flat_list.append(pred[1])
            

      
    return pred_list, buy_list, sell_list, flat_list

In [ ]:
# dataframe prep
df = pd.read_csv('')
df['startTime'] = pd.to_datetime(df['startTime'])
df.set_index('startTime', inplace=True)

day = 60*60*24
week = 60*60*24*7

df['Day_sin'] = np.sin(df['time'] * (2 * np.pi / day))
df['Day_cos'] = np.cos(df['time'] * (2 * np.pi / day))
df['Week_sin'] = np.sin(df['time'] * (2 * np.pi / week))
df['Week_cos'] = np.cos(df['time'] * (2 * np.pi / week))

df.drop(columns=['time'], inplace=True)
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)


model_path = '/model.model'


stats = {'Close': {'mean': 0.00013499451463623395, 'std': 0.010222616545887233},
         'High': {'mean': 0.00012189721875011916, 'std': 0.009010927715056327},
         'Low': {'mean': 0.00014861188842568972, 'std': 0.011457198801300696},
         'Open': {'mean': 0.0001339876181819151, 'std': 0.010219478504170286},
         'Volume': {'mean': 48690641.562491134, 'std': 78657724.08329752}}



# model_df
model1_df = df.copy()

In [ ]:
pred, buy_list, sell_list, flat_list = Neural_Network(model_df, model_path, stats, 20)


df = df.iloc[(len(df)-len(pred)):]
df['signal1'] = pred
df['buy_signal'] = buy_list
df['sell_signal'] = sell_list
df['flat_signal'] = flat_list
df['atr'] = ta.atr(high=df['High'], low=df['Low'], close=df['Close'], append=True)
df.dropna(inplace=True)
df

In [ ]:
def optimize_function(series):
    
    return (series['Profit Factor']*series['Sharpe Ratio'])/abs(series['Max. Drawdown [%]'])



class NeuralNet(Strategy):
    
    tp_factor = 3
    sl_factor = 1.5
    risk = 0.01


    def init(self):
        
        self.signal1 = self.I(SIGNAL1)
        self.atr = self.I(ATR)

    def next(self):
        
        
        tp_buy = self.data.Close[-1] + self.atr[-1]*self.tp_factor
        sl_buy = self.data.Close[-1] - self.atr[-1]*self.sl_factor
        tp_sell = self.data.Close[-1] - self.atr[-1]*self.tp_factor
        sl_sell = self.data.Close[-1] + self.atr[-1]*self.sl_factor
        acc_balance = self.equity
        
        position_size_buy = round((0.01*acc_balance)/(self.data.Close[-1] - sl_buy))        
        position_size_sell = round((0.01*acc_balance)/(sl_sell - self.data.Close[-1]))
        
        
        # Buy
        if self.signal1[-1]==2:
            if self.signal1[-2] != 2 and not self.position.is_long:
                self.position.close()
                self.buy(size=position_size_buy, 
                        tp=tp_buy, 
                        sl=sl_buy)


            
        # Sell
        if self.signal1[-1]==0:
            if self.signal1[-2] != 0 and not self.position.is_short:
                self.position.close()
                self.sell(size=position_size_sell,
                        tp=tp_sell, 
                        sl=sl_sell)
                


        # closing trades
        if  self.position.is_long and (self.signal1[-1]==0):
            self.position.close()

        if self.position.is_short and (self.signal1[-1]==2):
            self.position.close()
            
        

bt = Backtest(df, NeuralNet, cash=100_000, commission=0.0007)

stats = bt.run()

# Uncomment for optimisation of TP and SL points
# stats = bt.optimize(
#     tp_factor = [x / 10 for x in range(5, 50, 5)],
#     sl_factor = [x / 10 for x in range(5, 50, 5)],
#     maximize=optimize_function,
#     max_tries=500
# )

# print(stats)
# print(stats['_strategy'])

print(stats)
bt.plot()